In [8]:
import pandas as pd

file_path = "/Users/sagewong/Downloads/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/discharge.csv"
chunk_size = 1000
chunks = []

for chunk in pd.read_csv(file_path, chunksize = chunk_size):
    chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)

In [9]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331793 entries, 0 to 331792
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   note_id     331793 non-null  object
 1   subject_id  331793 non-null  int64 
 2   hadm_id     331793 non-null  int64 
 3   note_type   331793 non-null  object
 4   note_seq    331793 non-null  int64 
 5   charttime   331793 non-null  object
 6   storetime   331776 non-null  object
 7   text        331793 non-null  object
dtypes: int64(3), object(5)
memory usage: 20.3+ MB
None


In [10]:
a = df.loc[:, ['text']]
a['stigmatizing_language'] = ""


In [11]:
a.to_csv("mimic-iv-note-simplified.csv")

In [4]:
import pandas as pd
file_path = "mimic-iv-note-simplified.csv"
chunk_size = 1000
chunks = []

for chunk in pd.read_csv(file_path, chunksize = chunk_size):
    chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)
print(df.info)

<bound method DataFrame.info of         Unnamed: 0                                               text  \
0                0   \nName:  ___                     Unit No:   _...   
1                1   \nName:  ___                     Unit No:   _...   
2                2   \nName:  ___                     Unit No:   _...   
3                3   \nName:  ___                     Unit No:   _...   
4                4   \nName:  ___                    Unit No:   __...   
...            ...                                                ...   
331788      331788   \nName:  ___                   Unit No:   ___...   
331789      331789   \nName:  ___                   Unit No:   ___...   
331790      331790   \nName:  ___                  Unit No:   ___\...   
331791      331791   \nName:  ___                  Unit No:   ___\...   
331792      331792   \nName:  ___                    Unit No:   __...   

        stigmatizing_language  
0                         NaN  
1                         N

In [5]:
import tkinter as tk
from tkinter import scrolledtext
import csv
import pandas as pd

annotated_df = pd.read_csv("mimic-iv-note-simplified.csv")

dfIndexPos = 16

initialText = df.iloc[0]['stigmatizing_language']

root = tk.Tk()
root.title("Annotated dataset creator")
root.geometry("1200x900-100+0")

# Tkinter button functions
def nextButtonClicked():
    global dfIndexPos
    dfIndexPos += 1
    clinical_note_display.delete("1.0", tk.END)
    clinical_note_display.insert(tk.END, df.iloc[dfIndexPos]['text'])
    df.iloc[dfIndexPos]['stigmatizing_language'] = textbox_stigmatizing_language.get("1.0", "end-1c")
    textbox_stigmatizing_language.delete("1.0", tk.END)
    textbox_stigmatizing_language.insert(tk.END, df.iloc[dfIndexPos]['stigmatizing_language'])
    index_label.config(text=f"index: {dfIndexPos}")
    saveButton.config(bg="red")

def previousButtonClicked():
    global dfIndexPos
    if dfIndexPos > 0:
        dfIndexPos -= 1
    clinical_note_display.delete("1.0", tk.END)
    clinical_note_display.insert(tk.END, df.iloc[dfIndexPos]['text'])
    df.iloc[dfIndexPos]['stigmatizing_language'] = textbox_stigmatizing_language.get("1.0", "end-1c")
    textbox_stigmatizing_language.delete("1.0", tk.END)
    textbox_stigmatizing_language.insert(tk.END, df.iloc[dfIndexPos]['stigmatizing_language'])
    index_label.config(text=f"index: {dfIndexPos}")
    saveButton.config(bg="red")

def saveToDataset():
    new_df = pd.read_csv('mimic-iv-note-annotated-dataset.csv')
    new_row = pd.DataFrame([{'text': df.iloc[dfIndexPos]['text'], 'stigmatizing_language': textbox_stigmatizing_language.get("1.0", tk.END)}])
    new_df = pd.concat([new_df, new_row])
    new_df.to_csv('mimic-iv-note-annotated-dataset.csv',quoting=csv.QUOTE_ALL)
    saveButton.config(bg="green")

previousButton = tk.Button(root, text="previous cell", command=previousButtonClicked)
previousButton.grid(row=0, column=0)

clinical_note_display = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=100, height=50, font=("Arial", 12))
clinical_note_display.grid(row=0, column=1)
clinical_note_display.insert(tk.END, df.iloc[dfIndexPos]['text'])

index_label = tk.Label(root, text = f"index: {dfIndexPos}")
index_label.grid(row=1, column = 1)

textbox_stigmatizing_language = tk.Text(root, height=5, width=30)
textbox_stigmatizing_language.grid(row=2, column=1)
textbox_stigmatizing_language.insert("1.0", initialText)

nextButton = tk.Button(root, text="next cell", command=nextButtonClicked)
nextButton.grid(row=0, column=2)

saveButton = tk.Button(root, text="save to dataset", command=saveToDataset)
saveButton.grid(row=1, column = 2)

root.mainloop()

In [1]:
import tkinter as tk
from tkinter import scrolledtext
import pandas as pd
import re
import csv

annotated_df = pd.read_csv("mimic-iv-note-simplified.csv")

dfIndexPos = 16

initialText = annotated_df.iloc[0]['stigmatizing_language']

root = tk.Tk()
root.title("Annotated dataset creator")
root.geometry("1200x900-100+0")

highlight_words = ["failure", "narcotic"]

# Highlight function
def highlight_text():
    clinical_note_display.tag_remove("highlight", "1.0", tk.END)  # Remove existing highlights
    text_content = clinical_note_display.get("1.0", tk.END)
    
    for word in highlight_words:
        matches = re.finditer(rf"\b{word}\b", text_content, re.IGNORECASE)
        for match in matches:
            start = f"1.0+{match.start()}c"
            end = f"1.0+{match.end()}c"
            clinical_note_display.tag_add("highlight", start, end)
    
    clinical_note_display.tag_configure("highlight", background="yellow", foreground="black")

# Tkinter button functions
def nextButtonClicked():
    global dfIndexPos
    dfIndexPos += 1
    clinical_note_display.delete("1.0", tk.END)
    clinical_note_display.insert(tk.END, annotated_df.iloc[dfIndexPos]['text'])
    textbox_stigmatizing_language.delete("1.0", tk.END)
    textbox_stigmatizing_language.insert(tk.END, annotated_df.iloc[dfIndexPos]['stigmatizing_language'])
    index_label.config(text=f"index: {dfIndexPos}")
    saveButton.config(bg="red")
    highlight_text()

def previousButtonClicked():
    global dfIndexPos
    if dfIndexPos > 0:
        dfIndexPos -= 1
    clinical_note_display.delete("1.0", tk.END)
    clinical_note_display.insert(tk.END, annotated_df.iloc[dfIndexPos]['text'])
    textbox_stigmatizing_language.delete("1.0", tk.END)
    textbox_stigmatizing_language.insert(tk.END, annotated_df.iloc[dfIndexPos]['stigmatizing_language'])
    index_label.config(text=f"index: {dfIndexPos}")
    saveButton.config(bg="red")
    highlight_text()

def saveToDataset():
    new_df = pd.read_csv('mimic-iv-note-annotated-dataset.csv')
    new_row = pd.DataFrame([{'text': annotated_df.iloc[dfIndexPos]['text'], 'stigmatizing_language': textbox_stigmatizing_language.get("1.0", tk.END).strip()}])
    new_df = pd.concat([new_df, new_row], ignore_index=True)
    new_df.to_csv('mimic-iv-note-annotated-dataset.csv', quoting=csv.QUOTE_ALL, index=False)
    saveButton.config(bg="green")

previousButton = tk.Button(root, text="previous cell", command=previousButtonClicked)
previousButton.grid(row=0, column=0)

clinical_note_display = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=100, height=50, font=("Arial", 12))
clinical_note_display.grid(row=0, column=1)
clinical_note_display.insert(tk.END, annotated_df.iloc[dfIndexPos]['text'])
highlight_text()  # Initial highlight

index_label = tk.Label(root, text=f"index: {dfIndexPos}")
index_label.grid(row=1, column=1)

textbox_stigmatizing_language = tk.Text(root, height=5, width=30)
textbox_stigmatizing_language.grid(row=2, column=1)
textbox_stigmatizing_language.insert("1.0", initialText)

nextButton = tk.Button(root, text="next cell", command=nextButtonClicked)
nextButton.grid(row=0, column=2)

saveButton = tk.Button(root, text="save to dataset", command=saveToDataset)
saveButton.grid(row=1, column=2)

root.mainloop()


: 

In [ ]:
import tkinter as tk
from tkinter import scrolledtext
import pandas as pd
import re
import csv

annotated_df = pd.read_csv("mimic-iv-note-simplified.csv")

dfIndexPos = 16

initialText = annotated_df.iloc[0]['stigmatizing_language']

root = tk.Tk()
root.title("Annotated dataset creator")
root.geometry("1200x900-100+0")

highlight_words = ['adamant', 'adamantly', 'adament', 'adamently', 'claim', 'claimed', 'claiming', 'claims', 'insist', 'insisted', 'insistence', 'insisting', 'insists', 'adherance', 'adhere', 'adhered', 'adherence', 'adherent', 'adheres', 'adhering', 'compliance', 'compliant', 'complied', 'complies', 'comply', 'complying', 'declined', 'declines', 'declining', 'nonadherance', 'nonadherence', 'nonadherent', 'noncompliance', 'noncompliant', 'refusal', 'refuse', 'refused', 'refuses', 'refusing', 'aggression', 'aggressive', 'aggressively', 'agitated', 'agitation', 'anger', 'angered', 'angers', 'angrier', 'angrily', 'angry', 'argumentative', 'argumentatively', 'belligerence', 'belligerent', 'belligerently', 'charming', 'combative', 'combatively', 'confrontational', 'cooperative', 'defensive', 'delightful', 'disheveled', 'drug seeking', 'drug-seeking', 'exaggerate', 'exaggerates', 'exaggerating', 'historian', 'lovely', 'malinger', 'malingered', 'malingerer', 'malingering', 'malingers', 'narcotic seeking', 'narcotic-seeking', 'pleasant', 'pleasantly', 'poorly groomed', 'poorly-groomed', 'secondary gain', 'uncooperative', 'unkempt', 'unmotivated', 'unwilling', 'unwillingly', 'well groomed', 'well-groomed']

# Highlight function
def highlight_text():
    clinical_note_display.tag_remove("highlight", "1.0", tk.END)  # Remove existing highlights
    text_content = clinical_note_display.get("1.0", tk.END)
    
    for word in highlight_words:
        matches = re.finditer(rf"\b{word}\b", text_content, re.IGNORECASE)
        for match in matches:
            start = f"1.0+{match.start()}c"
            end = f"1.0+{match.end()}c"
            clinical_note_display.tag_add("highlight", start, end)
    
    clinical_note_display.tag_configure("highlight", background="yellow", foreground="black")

# Tkinter button functions
def update_display():
    clinical_note_display.delete("1.0", tk.END)
    clinical_note_display.insert(tk.END, annotated_df.iloc[dfIndexPos]['text'])
    highlight_text()
    textbox_stigmatizing_language.delete("1.0", tk.END)
    textbox_stigmatizing_language.insert(tk.END, annotated_df.iloc[dfIndexPos]['stigmatizing_language'])
    index_label.config(text=f"index: {dfIndexPos}")

def nextButtonClicked():
    global dfIndexPos
    dfIndexPos += 1
    update_display()
    saveButton.config(bg="red")

def previousButtonClicked():
    global dfIndexPos
    if dfIndexPos > 0:
        dfIndexPos -= 1
    update_display()
    saveButton.config(bg="red")

def jump_to_index():
    global dfIndexPos
    try:
        new_index = int(index_entry.get())
        if 0 <= new_index < len(annotated_df):
            dfIndexPos = new_index
            update_display()
        else:
            index_label.config(text="Invalid index")
    except ValueError:
        index_label.config(text="Enter a valid number")

def saveToDataset():
    new_df = pd.read_csv('mimic-iv-note-annotated-dataset.csv')
    new_row = pd.DataFrame([{'text': annotated_df.iloc[dfIndexPos]['text'], 'stigmatizing_language': textbox_stigmatizing_language.get("1.0", tk.END)}])
    new_df = pd.concat([new_df, new_row])
    new_df.to_csv('mimic-iv-note-annotated-dataset.csv', quoting=csv.QUOTE_ALL)
    saveButton.config(bg="green")

# UI Elements
previousButton = tk.Button(root, text="Previous Cell", command=previousButtonClicked)
previousButton.grid(row=0, column=0, padx=5, pady=5)

clinical_note_display = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=100, height=50, font=("Arial", 12))
clinical_note_display.grid(row=0, column=1, columnspan=2, padx=5, pady=5)
clinical_note_display.insert(tk.END, annotated_df.iloc[dfIndexPos]['text'])
highlight_text()

index_label = tk.Label(root, text=f"index: {dfIndexPos}")
index_label.grid(row=1, column=1, padx=5, pady=5)

textbox_stigmatizing_language = tk.Text(root, height=5, width=30)
textbox_stigmatizing_language.grid(row=2, column=1, columnspan=2, padx=5, pady=5)
textbox_stigmatizing_language.insert("1.0", initialText)

nextButton = tk.Button(root, text="Next Cell", command=nextButtonClicked)
nextButton.grid(row=0, column=3, padx=5, pady=5)

index_entry = tk.Entry(root, width=10)
index_entry.grid(row=1, column=0, padx=5, pady=5, sticky="w")

jumpButton = tk.Button(root, text="Jump to Index", command=jump_to_index)
jumpButton.grid(row=1, column=2, padx=5, pady=5, sticky="e")

saveButton = tk.Button(root, text="Save to Dataset", command=saveToDataset)
saveButton.grid(row=3, column=1, padx=5, pady=5)
root.mainloop()